
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img
    src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png"
    alt="Databricks Learning"
  >
</div>

# 3.1 DEMO: Cross Cloud Replication with Cloudflare R2 [Provider]

## Overview
This demo showcases how to set up cross-cloud replication using Cloudflare R2 as an intermediary storage layer. The provider creates a managed table with Change Data Feed (CDF) enabled, then replicates changes to an external table on R2 for global, cost-effective data sharing.

## Learning Objectives
By the end of this demo, you will understand:
1. How to enable Change Data Feed on managed tables
2. How to configure Cloudflare R2 external storage
3. How to replicate table changes to R2 using CDF
4. How to automate replication using Databricks Jobs

## Architecture
```
Managed Table (with CDF) → CDF Stream → External Table (R2) → Recipients
```

**Benefits:**
- Zero egress costs with Cloudflare R2
- Global data distribution without provider dependencies
- Automated change propagation using CDF
- Cost-effective sharing with unlimited recipients

## Setup

Run the common setup and demo configuration scripts.

In [ ]:
%run ./_common

In [ ]:
%run ./Demo-Setup-3_1

## Step 1: Configure Cloudflare R2 Storage Credentials

First, we need to configure the storage credentials for Cloudflare R2. In production, you would store these as secrets.

In [ ]:
# R2 Configuration - Replace with your actual values
# In production, use Databricks Secrets for these values
r2_endpoint = "https://4132d7d5587ee99b9d482ecfc2c1853c.r2.cloudflarestorage.com"
r2_bucket = "databricks-demo"
r2_access_key = "<your-r2-access-key>"  # Replace with actual key
r2_secret_key = "<your-r2-secret-key>"  # Replace with actual key

# Construct the full R2 path for our replica table
r2_table_path = f"s3a://{r2_bucket}/{DA.r2_path}"

print(f"R2 Endpoint: {r2_endpoint}")
print(f"R2 Bucket: {r2_bucket}")
print(f"R2 Table Path: {r2_table_path}")

## Step 2: Configure Spark for R2 Access

Configure Spark to use the R2 credentials and endpoint.

In [ ]:
# Configure Spark for R2 access
spark.conf.set(f"fs.s3a.bucket.{r2_bucket}.endpoint", r2_endpoint)
spark.conf.set(f"fs.s3a.bucket.{r2_bucket}.access.key", r2_access_key)
spark.conf.set(f"fs.s3a.bucket.{r2_bucket}.secret.key", r2_secret_key)
spark.conf.set(f"fs.s3a.bucket.{r2_bucket}.path.style.access", "true")
spark.conf.set(f"fs.s3a.bucket.{r2_bucket}.connection.ssl.enabled", "true")

print("✅ Spark configured for R2 access")

## Step 3: Create Source Managed Table with Sample Data

Create a managed table that will serve as our primary data source.

In [ ]:
# Create the source transactions table with initial data
spark.sql(f"""
CREATE OR REPLACE TABLE {DA.catalog}.{DA.schema}.{DA.source_table} (
  transaction_id STRING,
  customer_id STRING,
  product_category STRING,
  amount DECIMAL(10,2),
  transaction_date DATE,
  region STRING,
  created_at TIMESTAMP
)
USING DELTA
PARTITIONED BY (transaction_date)
TBLPROPERTIES (
  'delta.enableChangeDataFeed' = 'true',
  'delta.autoOptimize.optimizeWrite' = 'true',
  'delta.autoOptimize.autoCompact' = 'true'
)
""")

print(f"✅ Created managed table: {DA.catalog}.{DA.schema}.{DA.source_table}")

In [ ]:
# Insert initial sample data
from datetime import datetime, date
import uuid

# Generate sample transactions
sample_data = []
regions = ['North America', 'Europe', 'Asia Pacific', 'Latin America']
categories = ['Electronics', 'Clothing', 'Books', 'Home & Garden', 'Sports']

for i in range(100):
    sample_data.append((
        str(uuid.uuid4()),
        f"customer_{i % 50}",
        categories[i % len(categories)],
        round(50 + (i * 3.7) % 500, 2),
        date(2024, 10, 20 + (i % 7)),
        regions[i % len(regions)],
        datetime.now()
    ))

# Create DataFrame and insert
from pyspark.sql.types import StructType, StructField, StringType, DecimalType, DateType, TimestampType

schema = StructType([
    StructField("transaction_id", StringType(), True),
    StructField("customer_id", StringType(), True),
    StructField("product_category", StringType(), True),
    StructField("amount", DecimalType(10,2), True),
    StructField("transaction_date", DateType(), True),
    StructField("region", StringType(), True),
    StructField("created_at", TimestampType(), True)
])

df = spark.createDataFrame(sample_data, schema)
df.write.mode("append").saveAsTable(f"{DA.catalog}.{DA.schema}.{DA.source_table}")

print(f"✅ Inserted {df.count()} sample transactions")

## Step 4: Verify Change Data Feed is Enabled

Check that CDF is properly enabled on our source table.

In [ ]:
# Check table properties to confirm CDF is enabled
table_details = spark.sql(f"DESCRIBE DETAIL {DA.catalog}.{DA.schema}.{DA.source_table}")
display(table_details.select("name", "location", "properties"))

# Get current version
current_version = table_details.select("version").collect()[0][0]
print(f"\n✅ Current table version: {current_version}")
print(f"✅ CDF enabled: Look for 'delta.enableChangeDataFeed' = 'true' in properties")

## Step 5: Create External Table on Cloudflare R2

Create an external table that will store our replicated data on R2.

In [ ]:
# Create external table on R2 with same schema
spark.sql(f"""
CREATE OR REPLACE TABLE {DA.catalog}.{DA.schema}.{DA.replica_table} (
  transaction_id STRING,
  customer_id STRING,
  product_category STRING,
  amount DECIMAL(10,2),
  transaction_date DATE,
  region STRING,
  created_at TIMESTAMP
)
USING DELTA
LOCATION '{r2_table_path}'
PARTITIONED BY (transaction_date)
TBLPROPERTIES (
  'delta.autoOptimize.optimizeWrite' = 'true',
  'delta.autoOptimize.autoCompact' = 'true'
)
""")

print(f"✅ Created external table on R2: {DA.catalog}.{DA.schema}.{DA.replica_table}")
print(f"✅ Location: {r2_table_path}")

## Step 6: Initial Data Replication

Perform the initial full copy of data to the R2 external table.

In [ ]:
# Perform initial full replication
source_df = spark.table(f"{DA.catalog}.{DA.schema}.{DA.source_table}")

# Write to external R2 table
source_df.write \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(f"{DA.catalog}.{DA.schema}.{DA.replica_table}")

print(f"✅ Initial replication completed")
print(f"   Source records: {source_df.count()}")

# Verify replication
replica_count = spark.table(f"{DA.catalog}.{DA.schema}.{DA.replica_table}").count()
print(f"   Replica records: {replica_count}")

## Step 7: Add More Data to Source Table

Simulate ongoing business operations by adding more data to the source table.

In [ ]:
# Add new transactions to simulate ongoing operations
new_data = []
for i in range(100, 150):
    new_data.append((
        str(uuid.uuid4()),
        f"customer_{i % 60}",
        categories[i % len(categories)],
        round(75 + (i * 4.2) % 400, 2),
        date(2024, 10, 27),  # Today's date
        regions[i % len(regions)],
        datetime.now()
    ))

new_df = spark.createDataFrame(new_data, schema)
new_df.write.mode("append").saveAsTable(f"{DA.catalog}.{DA.schema}.{DA.source_table}")

print(f"✅ Added {new_df.count()} new transactions")

# Check new version
new_version = spark.sql(f"DESCRIBE DETAIL {DA.catalog}.{DA.schema}.{DA.source_table}").select("version").collect()[0][0]
print(f"✅ Table version updated to: {new_version}")

## Step 8: Read Change Data Feed

Read the changes from the source table using Change Data Feed.

In [ ]:
# Read changes from the last version
changes_df = spark.read \
    .format("delta") \
    .option("readChangeDataFeed", "true") \
    .option("startingVersion", current_version + 1) \
    .table(f"{DA.catalog}.{DA.schema}.{DA.source_table}")

print(f"✅ Changes detected: {changes_df.count()} records")

# Show the structure of CDF data
print("\n📊 Change Data Feed Structure:")
changes_df.printSchema()

# Display sample changes
print("\n📄 Sample Changes:")
display(changes_df.select("transaction_id", "amount", "region", "_change_type", "_commit_version").limit(10))

## Step 9: Apply Changes to R2 Replica

Apply the detected changes to our R2 external table.

In [ ]:
# Filter only the data changes (ignore metadata)
data_changes = changes_df.filter("_change_type = 'insert'")

if data_changes.count() > 0:
    # Select only the business columns (exclude CDF metadata)
    business_data = data_changes.select(
        "transaction_id", "customer_id", "product_category", 
        "amount", "transaction_date", "region", "created_at"
    )
    
    # Append changes to R2 table
    business_data.write \
        .mode("append") \
        .saveAsTable(f"{DA.catalog}.{DA.schema}.{DA.replica_table}")
    
    print(f"✅ Applied {business_data.count()} changes to R2 replica")
else:
    print("ℹ️  No data changes to apply")

# Verify replication is in sync
source_count = spark.table(f"{DA.catalog}.{DA.schema}.{DA.source_table}").count()
replica_count = spark.table(f"{DA.catalog}.{DA.schema}.{DA.replica_table}").count()

print(f"\n📊 Replication Status:")
print(f"   Source table: {source_count} records")
print(f"   R2 replica:   {replica_count} records")
print(f"   In sync: {'✅ Yes' if source_count == replica_count else '❌ No'}")

## Step 10: Create Replication Function

Create a reusable function for ongoing replication that can be scheduled as a job.

In [ ]:
def replicate_changes_to_r2(source_table, replica_table, last_processed_version=None):
    """
    Replicate changes from source table to R2 replica using Change Data Feed.
    
    Args:
        source_table: Fully qualified source table name
        replica_table: Fully qualified replica table name
        last_processed_version: Last version that was processed (optional)
    
    Returns:
        dict: Statistics about the replication process
    """
    try:
        # Get current version of source table
        current_version = spark.sql(f"DESCRIBE DETAIL {source_table}").select("version").collect()[0][0]
        
        # If no last_processed_version provided, get it from replica table properties
        if last_processed_version is None:
            try:
                replica_props = spark.sql(f"DESCRIBE DETAIL {replica_table}").select("properties").collect()[0][0]
                last_processed_version = int(replica_props.get('last_processed_version', '0'))
            except:
                last_processed_version = 0
        
        starting_version = last_processed_version + 1
        
        print(f"🔄 Checking for changes from version {starting_version} to {current_version}")
        
        if starting_version > current_version:
            return {
                'status': 'up_to_date',
                'records_processed': 0,
                'current_version': current_version,
                'message': 'Replica is up to date'
            }
        
        # Read changes using CDF
        changes_df = spark.read \
            .format("delta") \
            .option("readChangeDataFeed", "true") \
            .option("startingVersion", starting_version) \
            .table(source_table)
        
        # Process different change types
        inserts = changes_df.filter("_change_type = 'insert'")
        updates_pre = changes_df.filter("_change_type = 'update_preimage'")
        updates_post = changes_df.filter("_change_type = 'update_postimage'")
        deletes = changes_df.filter("_change_type = 'delete'")
        
        total_changes = 0
        
        # Apply inserts
        if inserts.count() > 0:
            business_data = inserts.select(
                "transaction_id", "customer_id", "product_category",
                "amount", "transaction_date", "region", "created_at"
            )
            business_data.write.mode("append").saveAsTable(replica_table)
            insert_count = business_data.count()
            total_changes += insert_count
            print(f"   ✅ Applied {insert_count} inserts")
        
        # For this demo, we'll focus on inserts. In production, you'd handle
        # updates and deletes using MERGE statements
        
        # Update replica table properties with last processed version
        spark.sql(f"""
            ALTER TABLE {replica_table} 
            SET TBLPROPERTIES ('last_processed_version' = '{current_version}')
        """)
        
        return {
            'status': 'success',
            'records_processed': total_changes,
            'current_version': current_version,
            'starting_version': starting_version,
            'message': f'Successfully replicated {total_changes} changes'
        }
        
    except Exception as e:
        return {
            'status': 'error',
            'records_processed': 0,
            'error': str(e),
            'message': f'Replication failed: {str(e)}'
        }

print("✅ Replication function created")

## Step 11: Test the Replication Function

Test our replication function with additional data changes.

In [ ]:
# Add more test data
test_data = []
for i in range(200, 220):
    test_data.append((
        str(uuid.uuid4()),
        f"customer_{i % 70}",
        categories[i % len(categories)],
        round(100 + (i * 2.5) % 300, 2),
        date(2024, 10, 27),
        regions[i % len(regions)],
        datetime.now()
    ))

test_df = spark.createDataFrame(test_data, schema)
test_df.write.mode("append").saveAsTable(f"{DA.catalog}.{DA.schema}.{DA.source_table}")

print(f"✅ Added {test_df.count()} test transactions")

# Test the replication function
result = replicate_changes_to_r2(
    f"{DA.catalog}.{DA.schema}.{DA.source_table}",
    f"{DA.catalog}.{DA.schema}.{DA.replica_table}"
)

print(f"\n📊 Replication Result:")
print(f"   Status: {result['status']}")
print(f"   Records processed: {result['records_processed']}")
print(f"   Message: {result['message']}")

## Step 12: Verify Replication and Data Quality

Perform final verification that our replication is working correctly.

In [ ]:
# Compare source and replica data
source_stats = spark.sql(f"""
    SELECT 
        COUNT(*) as total_records,
        COUNT(DISTINCT transaction_id) as unique_transactions,
        ROUND(SUM(amount), 2) as total_amount,
        MAX(transaction_date) as latest_date
    FROM {DA.catalog}.{DA.schema}.{DA.source_table}
""").collect()[0]

replica_stats = spark.sql(f"""
    SELECT 
        COUNT(*) as total_records,
        COUNT(DISTINCT transaction_id) as unique_transactions,
        ROUND(SUM(amount), 2) as total_amount,
        MAX(transaction_date) as latest_date
    FROM {DA.catalog}.{DA.schema}.{DA.replica_table}
""").collect()[0]

print("📊 Data Quality Verification:")
print(f"\n   Source Table ({DA.source_table}):")
print(f"     Records: {source_stats[0]}")
print(f"     Unique transactions: {source_stats[1]}")
print(f"     Total amount: ${source_stats[2]}")
print(f"     Latest date: {source_stats[3]}")

print(f"\n   R2 Replica ({DA.replica_table}):")
print(f"     Records: {replica_stats[0]}")
print(f"     Unique transactions: {replica_stats[1]}")
print(f"     Total amount: ${replica_stats[2]}")
print(f"     Latest date: {replica_stats[3]}")

# Verify data integrity
data_match = (
    source_stats[0] == replica_stats[0] and
    source_stats[1] == replica_stats[1] and
    source_stats[2] == replica_stats[2]
)

print(f"\n   Data Integrity: {'✅ PASSED' if data_match else '❌ FAILED'}")

if data_match:
    print("   🎉 Replication is working perfectly!")
else:
    print("   ⚠️  Data mismatch detected - check replication logic")

## Step 13: Create Job-Ready Replication Script

Create a production-ready script that can be scheduled as a Databricks Job.

In [ ]:
# Production replication script
def production_replication_job():
    """
    Production-ready replication job that can be scheduled.
    This function includes error handling, logging, and monitoring.
    """
    import json
    from datetime import datetime
    
    job_start = datetime.now()
    
    try:
        print(f"🔄 Starting replication job at {job_start}")
        
        # Define table names (in production, these could be parameters)
        source_table = f"{DA.catalog}.{DA.schema}.{DA.source_table}"
        replica_table = f"{DA.catalog}.{DA.schema}.{DA.replica_table}"
        
        # Run replication
        result = replicate_changes_to_r2(source_table, replica_table)
        
        job_end = datetime.now()
        duration = (job_end - job_start).total_seconds()
        
        # Log results
        log_entry = {
            'timestamp': job_start.isoformat(),
            'duration_seconds': duration,
            'status': result['status'],
            'records_processed': result['records_processed'],
            'source_table': source_table,
            'replica_table': replica_table
        }
        
        print(f"\n✅ Job completed successfully")
        print(f"   Duration: {duration:.2f} seconds")
        print(f"   Records processed: {result['records_processed']}")
        print(f"   Status: {result['status']}")
        
        # In production, you might want to:
        # 1. Send metrics to monitoring system
        # 2. Log to centralized logging system
        # 3. Send alerts on failure
        # 4. Update job status in metadata table
        
        return log_entry
        
    except Exception as e:
        job_end = datetime.now()
        duration = (job_end - job_start).total_seconds()
        
        error_log = {
            'timestamp': job_start.isoformat(),
            'duration_seconds': duration,
            'status': 'error',
            'error_message': str(e),
            'source_table': source_table,
            'replica_table': replica_table
        }
        
        print(f"❌ Job failed after {duration:.2f} seconds")
        print(f"   Error: {str(e)}")
        
        # In production, send alert to operations team
        raise

# Test the production job
job_result = production_replication_job()
print(f"\n📋 Job Result: {job_result}")

## Step 14: Monitoring and Observability

Set up monitoring to track replication performance and health.

In [ ]:
# Create monitoring view for replication status
spark.sql(f"""
CREATE OR REPLACE VIEW {DA.catalog}.{DA.schema}.replication_status AS
SELECT 
    '{DA.source_table}' as source_table,
    '{DA.replica_table}' as replica_table,
    '{r2_table_path}' as r2_location,
    source.record_count as source_records,
    replica.record_count as replica_records,
    source.latest_version as source_version,
    replica.last_processed_version,
    CASE 
        WHEN source.record_count = replica.record_count THEN 'IN_SYNC'
        ELSE 'OUT_OF_SYNC'
    END as sync_status,
    current_timestamp() as check_time
FROM (
    SELECT 
        COUNT(*) as record_count,
        MAX(version) as latest_version
    FROM (
        SELECT version FROM (
            DESCRIBE DETAIL {DA.catalog}.{DA.schema}.{DA.source_table}
        )
    ) v
    CROSS JOIN (
        SELECT COUNT(*) as cnt FROM {DA.catalog}.{DA.schema}.{DA.source_table}
    ) c
) source
CROSS JOIN (
    SELECT 
        COUNT(*) as record_count,
        COALESCE(props.last_processed_version, '0') as last_processed_version
    FROM (
        SELECT COUNT(*) as cnt FROM {DA.catalog}.{DA.schema}.{DA.replica_table}
    ) c
    CROSS JOIN (
        SELECT properties['last_processed_version'] as last_processed_version
        FROM (
            DESCRIBE DETAIL {DA.catalog}.{DA.schema}.{DA.replica_table}
        )
    ) props
) replica
""")

print("✅ Created replication monitoring view")

# Check current status
print("\n📊 Current Replication Status:")
display(spark.sql(f"SELECT * FROM {DA.catalog}.{DA.schema}.replication_status"))

## Summary and Next Steps

🎉 **Congratulations!** You have successfully set up cross-cloud replication with Cloudflare R2.

### What We Accomplished:

✅ **Source Setup**: Created a managed table with Change Data Feed enabled  
✅ **R2 Configuration**: Set up Cloudflare R2 external storage with S3-compatible API  
✅ **Initial Replication**: Performed full data copy to R2 external table  
✅ **Change Detection**: Used CDF to detect and capture data changes  
✅ **Incremental Updates**: Applied changes to R2 replica automatically  
✅ **Production Function**: Created job-ready replication function  
✅ **Monitoring**: Set up observability for replication status  

### Key Benefits Achieved:

🌍 **Global Distribution**: Data available worldwide via Cloudflare's network  
💰 **Zero Egress Costs**: Eliminate expensive data transfer fees  
⚡ **High Performance**: Fast access through global CDN  
🔄 **Automated Sync**: Real-time change propagation using CDF  
📊 **Monitoring**: Full observability into replication health  

### Next Steps for Production:

1. **Schedule the Job**: Set up the replication function as a Databricks Job (hourly/daily)
2. **Add Secrets**: Store R2 credentials in Databricks Secrets
3. **Error Handling**: Implement retry logic and alerting
4. **Performance Tuning**: Optimize for your data volume and frequency
5. **Security**: Configure appropriate access controls and encryption
6. **Monitoring**: Set up dashboards and alerts for replication health

### Scheduling as a Job:

To schedule this as a Databricks Job:

1. Create a new job in Databricks Workflows
2. Use this notebook or create a dedicated script
3. Set appropriate cluster configuration
4. Configure schedule (e.g., every 15 minutes)
5. Add email alerts for failures

The recipients can now access this data through Delta Sharing with zero egress costs!

---
&copy; 2025 Databricks, Inc. All rights reserved. Apache, Apache Spark, Spark, the Spark Logo, Apache Iceberg, Iceberg, and the Apache Iceberg logo are trademarks of the <a href="https://www.apache.org/" target="_blank">Apache Software Foundation</a>.<br/><br/><a href="https://databricks.com/privacy-policy" target="_blank">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use" target="_blank">Terms of Use</a> | <a href="https://help.databricks.com/" target="_blank">Support</a>